In [ ]:
!nvidia-smi

In [1]:
################################################################################
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "7"
import importlib
import numpy as np
import pandas as pd
import json
import sys
from IPython.display import display
from typing import Iterable, List, Sequence

In [2]:
################################################################################
def flatten(
        iterable: Iterable
        ) -> Iterable:
    """ Return a flattened iterable from a nested iterable.
        [[3, [4, 5]], 6, [[[7]]]] -> [3, 4, 5, 6, 7]
    """
    for item in iterable:
        if  isinstance(item, Iterable) and not isinstance(item, (str, bytes)):
            yield from flatten(item)
        else:
            yield item


def parse_file(
        filename         : str,
        initial_event_id : int      = 0,
        ignored_columns  : Sequence = (),
        ) -> Iterable[Iterable]:
    """ Parses the lines in the file from 'filename' to a format
        appropriate for passing into a pandas DataFrame constructor.
    """
    event_id = initial_event_id
    with open(filename) as file:
        lines = filter(None, (line.strip() for line in file))
        for line in lines:
            if line.startswith("#"):
                event_id += 1
            else:
                j_list = json.loads("[{0}]".format(line))
                for column in ignored_columns:
                    del j_list[column]
                j_list.append(event_id)
                yield flatten(j_list)

In [5]:
################################################################################
clusters_columns = [
    "hit_nr", "barcode", "volume_id", "layer_id",
    "lx",     "ly",      "elx",       "ely",    
    "gx",     "gy",      "gz",        "phi",    
    "theta",  "ephi",    "etheta",    "event_id",
]
particles_columns = [
    "barcode",  "vertex_x", "vertex_y",
    "vertex_z", "momentum", "theta",
    "phi",      "charge",   "event_id",
]

# Extraction from a single file.

In [ ]:
################################################################################
number = 1
base_directory     = "/inputdata/ACTS/prod_mu10_pt1000_2017_07_29"
clusters_filename  = base_directory + "/clusters_{0}.csv".format(number)
particles_filename = base_directory + "/particles_{0}.csv".format(number)

In [ ]:
################################################################################
clusters_lines = parse_file(clusters_filename, ignored_columns=[7])
clusters_frame = pd.DataFrame(clusters_lines, columns=clusters_columns)
clusters_frame.head(3)

In [ ]:
################################################################################
particles_lines = parse_file(particles_filename)
particles_frame = pd.DataFrame(particles_lines, columns=particles_columns)
particles_frame.head(3)

In [ ]:
################################################################################
left_frame     = clusters_frame
right_frame    = particles_frame[["event_id", "barcode", "momentum", "charge"]]
combined_frame = left_frame.merge(right_frame, on=["event_id", "barcode"])
combined_frame.head(3)

In [ ]:
################################################################################
gx    = combined_frame["gx"]
gy    = combined_frame["gy"]
gz    = combined_frame["gz"]
phi   = np.arctan2(gy, gx)
r     = np.sqrt(gx**2 + gy**2)
frame = combined_frame.assign(phi=phi, r=r, z=gz)

In [ ]:
################################################################################
# Eliminate duplicate hits that were caused by imperfections in the detector.
frame = frame.sort_values("r")
frame = frame.drop_duplicates(["event_id", "barcode", "layer_id"])

In [ ]:
################################################################################
# Specify the volume to use. Each volume is a different detector configuration.
frame = frame[frame["volume_id"] == 8]

In [ ]:
################################################################################
# Set radiuses to be the same for each layer.
for layer_id in frame["layer_id"].unique():
    ind = frame["layer_id"] == layer_id
    rs  = frame[ind]["r"]
    med = rs.median()
    frame.loc[ind, "r"] = med

In [ ]:
################################################################################
# Put limits on the number of tracks per event. 
max_tracks = 50
min_tracks = 2
frames = [f for (_, f) in frame.groupby("event_id", sort=False)]
for i, f in enumerate(frames):
    barcodes = f["barcode"].unique()
    if len(barcodes) < min_tracks:
        frames[i] = pd.DataFrame()
    if len(barcodes) > max_tracks:
        length = np.random.randint(min_tracks, max_tracks + 1)
        barcodes = np.random.choice(barcodes, length, replace=False)
        f = f[f["barcode"].isin(barcodes)]
        frames[i] = f
frame = pd.concat(frames)

In [ ]:
################################################################################
# Clean up the frame a bit.
frame = frame[["event_id", "barcode", "phi", "r", "z", "momentum", "charge"]]
frame = frame.sort_values(["event_id", "barcode", "r"])
frame.head(3)

# Extraction from multiple files.

In [6]:
################################################################################
def extract(
        clusters_filename  : str, 
        particles_filename : str, 
        min_tracks         : int = 2, 
        max_tracks         : int = 50,
        initial_event_id   : int = 0,
        ) -> pd.DataFrame:
    """ Everything in one function.
        Depending on the size of the file, this function could take a long
        time. Most of the time is spent parsing the csv files within the
        first 4 lines.
    """
    clusters_lines = parse_file(
        clusters_filename,
        ignored_columns=[7],
        initial_event_id=initial_event_id)
    clusters_frame = pd.DataFrame(clusters_lines, columns=clusters_columns)
    particles_lines = parse_file(
        particles_filename, 
        initial_event_id=initial_event_id)
    particles_frame = pd.DataFrame(particles_lines, columns=particles_columns)
    left  = clusters_frame
    right = particles_frame[["event_id", "barcode", "momentum", "charge"]]
    combined_frame = left.merge(right, on=["event_id", "barcode"])
    gx    = combined_frame["gx"]
    gy    = combined_frame["gy"]
    gz    = combined_frame["gz"]
    phi   = np.arctan2(gy, gx)
    r     = np.sqrt(gx**2 + gy**2)
    frame = combined_frame.assign(phi=phi, r=r, z=gz)
    frame = frame.sort_values("r")
    frame = frame.drop_duplicates(["event_id", "barcode", "layer_id"])
    frame = frame[frame["volume_id"] == 8]
    for layer_id in frame["layer_id"].unique():
        ind = frame["layer_id"] == layer_id
        rs  = frame[ind]["r"]
        med = rs.median()
        frame.loc[ind, "r"] = med
    frames = [f for (_, f) in frame.groupby("event_id", sort=False)]
    for i, f in enumerate(frames):
        barcodes = f["barcode"].unique()
        if len(barcodes) < min_tracks:
            frames[i] = pd.DataFrame()
        if len(barcodes) > max_tracks:
            length = np.random.randint(min_tracks, max_tracks + 1)
            barcodes = np.random.choice(barcodes, length, replace=False)
            f = f[f["barcode"].isin(barcodes)]
            frames[i] = f
    frame = pd.concat(frames)
    frame = frame[["event_id", "barcode", "phi", "r",
                   "z", "momentum", "charge"]]
    frame = frame.sort_values(["event_id", "barcode", "r"])
    return frame

In [7]:
%%time
################################################################################
frames = []
initial_event_id = 0
for i in range(1, 1 + 100):
    print("Extracting from file {0}".format(i))
    try:
        base_directory     = "/inputdata/ACTS/prod_mu10_pt1000_2017_07_29"
        clusters_filename  = base_directory + "/clusters_{0}.csv".format(i)
        particles_filename = base_directory + "/particles_{0}.csv".format(i)
        frame = extract(
            clusters_filename=clusters_filename, 
            particles_filename=particles_filename, 
            min_tracks=2, 
            max_tracks=50, 
            initial_event_id=initial_event_id)
        initial_event_id = frame["event_id"].max() + 1
        frames.append(frame)
    except FileNotFoundError as error:
        print(error)
frame = pd.concat(frames)

Extracting from file 1
Extracting from file 2
169874
CPU times: user 1min 56s, sys: 716 ms, total: 1min 57s
Wall time: 1min 57s


In [12]:
print("Number of Hits: {}".format(len(frame)))
print("Number of Events: {}".format(len(frame["event_id"].unique())))
tracks  = [value for (_, value) in frame.groupby(["event_id"])]
lengths = [len(value["barcode"].unique()) for value in tracks]
print("Min Number of Tracks: {}".format(min(lengths)))
print("Max Number of Tracks: {}".format(max(lengths)))

Number of Hits: 169874
Number of Events: 1983
Min Number of Tracks: 2
Max Number of Tracks: 50


In [16]:
filepath = "data/sets/ACTS-MOMENTUMS.gz"
frame.to_csv(filepath, compression="gzip")
print("{0} bytes".format(os.path.getsize(filepath)))

3954909 bytes
